<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/Development/District-Builtup-Index-GHSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

[ee.ImageCollection.aggregate_array](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-aggregate_array):  Aggregates over a given property of the objects in a collection, calculating a list of all the values of the selected property.

In [75]:
# Load the district boundaries dataset
districts = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM0_NAME', 'India')).select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area'])
districts_dict = districts.select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area']).first().toDictionary().getInfo()
feature = districts.first()

## Load the district geometries

In [99]:
%%time
gdf = geemap.ee_to_gdf(districts.select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area']))

geo_df_simplified = gdf.copy()

## Convert the coordinates to meters
geo_df_simplified = geo_df_simplified.to_crs("EPSG:3857")
## Simplify the geometry by tolerance distance: 1 KM
geo_df_simplified['geometry'] = geo_df_simplified['geometry'].simplify(tolerance=1000,preserve_topology=True)

## Reproject to WGS:84
geo_df_simplified = geo_df_simplified.to_crs("EPSG:4326")

memory_usuage = geo_df_simplified.memory_usage(deep=True).sum()/1024
print(f"Memory usuage of this dataframe is {memory_usuage:.2f} KB")

print(geo_df_simplified.shape)
geo_df_simplified.head()

Memory usuage of this dataframe is 87.48 KB
(573, 5)
CPU times: user 4.08 s, sys: 158 ms, total: 4.24 s
Wall time: 13.6 s


,geometry,ADM1_NAME,ADM2_CODE,ADM2_NAME,Shape_Area
0,"POLYGON ((76.48194 29.51891, 76.49844 29.48948...",Haryana,17660,Karnal,0.240933
1,"POLYGON ((76.4125 30.04788, 76.4283 30.02573, ...",Haryana,17661,Kurukshetra,0.171739
2,"POLYGON ((75.889 28.39892, 75.89713 28.38008, ...",Haryana,17662,Mahendragarh,0.177959
3,"POLYGON ((76.46556 29.24227, 76.50406 29.16966...",Haryana,17665,Sonepat,0.205076
4,"POLYGON ((76.16629 29.92866, 76.18199 29.92324...",Haryana,70134,Kaithal,0.214695


## District wise built up area
Load the image collection containing the built up area

In [ ]:
image_collection = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")

years = image_collection.aggregate_array('system:index').getInfo()
print(years)

- calculate the total builtup area for each district year wise
- export the results

In [83]:
%%time

# Load the GHSL built-up surface dataset
ghsl_built_up = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")

# Function to calculate the built-up surface area for each district for a given year
def calculate_built_up_area(feature, year):

    district_id = feature.get('ADM2_CODE')

    built_up_image = ghsl_built_up.filterDate(f'{year}-01-01', f'{year}-12-31').first()

    built_up_image = built_up_image

    built_up_area = built_up_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=100,  # Scale to match GHSL data resolution
        maxPixels=1e9
    ).get('built_surface')

    return feature.set({'district_id':district_id, 'year': year, 'area_m2': built_up_area})

# Initialize an empty list to store results
results_list = []


properties = ['district_id', 'area_m2', 'year']
dfs = []
# Loop through each year and calculate the built-up area for each district
progress_bar1 = tqdm(years, unit="Year", colour='green')
for year in progress_bar1:
    progress_bar1.set_description(f"Processing year: {year}")
    districts_with_built_up_area = districts.map(lambda feature: calculate_built_up_area(feature, year))
    data = {}
    progress_bar2 = tqdm(properties, unit="property", leave=False, colour='blue')
    for prop in progress_bar2:
        progress_bar2.set_description(f"Processing property: {prop}")
        values = districts_with_built_up_area.aggregate_array(prop).getInfo()
        data[prop] = values

    df = pd.DataFrame(data)
    dfs.append(df)

  0%|          | 0/12 [00:00<?, ?Year/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

...

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

CPU times: user 2.19 s, sys: 209 ms, total: 2.4 s
Wall time: 4min 5s


 ## Concat the results

In [93]:
df_final = pd.concat(dfs)
df_final['year'] = df_final['year'].astype(int)
df_final.head()

,district_id,area_m2,year
0,17660,1.724510e+07,1975
1,17661,1.194741e+07,1975
2,17662,5.877115e+06,1975
3,17665,1.850835e+07,1975
4,70134,1.140200e+07,1975


## Merge the results for one year

In [98]:
xdf = df_final.loc[(df_final['year']==2005)]
xdf2 = pd.merge(
    left = xdf,
    right = gdf,
    how='left',
    left_on = 'district_id',
    right_on = 'ADM2_CODE'
)
xdf2

,district_id,area_m2,year,geometry,ADM1_NAME,ADM2_CODE,ADM2_NAME,Shape_Area
0,17660,4.568129e+07,2005,"POLYGON ((76.48194 29.51891, 76.48206 29.51872...",Haryana,17660,Karnal,0.240933
1,17661,3.489666e+07,2005,"POLYGON ((76.4125 30.04788, 76.41295 30.046, 7...",Haryana,17661,Kurukshetra,0.171739
2,17662,3.237908e+07,2005,"POLYGON ((75.889 28.39892, 75.88923 28.39366, ...",Haryana,17662,Mahendragarh,0.177959
3,17665,4.639188e+07,2005,"POLYGON ((76.46556 29.24227, 76.47164 29.23409...",Haryana,17665,Sonepat,0.205076
4,70134,3.652202e+07,2005,"POLYGON ((76.16629 29.92866, 76.17149 29.92323...",Haryana,70134,Kaithal,0.214695
...,...,...,...,...,...,...,...,...
568,70171,1.255956e+07,2005,"POLYGON ((75.94832 21.48328, 75.95609 21.48341...",Madhya Pradesh,70171,Burhanpur,0.278397
569,70226,5.174163e+06,2005,"POLYGON ((79.70287 10.94562, 79.70324 10.93829...",Puducherry,70226,Karaikal,0.016047
570,70227,5.530427e+06,2005,"POLYGON ((75.22113 12.10095, 75.22246 12.08557...",Puducherry,70227,Mahe,0.011121
571,70228,1.085624e+07,2005,"POLYGON ((79.64234 11.97581, 79.6442 11.97321,...",Puducherry,70228,Puducherry,0.016531
